In [1]:
import numpy as np
import torch
from torch import nn, optim
import tqdm
import torchvision
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split

In [2]:
# Use the following code to load and normalize the dataset for training and testing
# It will downlad the dataset into data subfolder (change to your data folder name)
train_dataset = torchvision.datasets.FashionMNIST('data/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

test_dataset = torchvision.datasets.FashionMNIST('data/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ]))

# Use the following code to create a validation set of 10%
train_indices, val_indices, _, _ = train_test_split(
    range(len(train_dataset)),
    train_dataset.targets,
    stratify=train_dataset.targets,
    test_size=0.1,
)

# Generate training and validation subsets based on indices
train_split = Subset(train_dataset, train_indices)
val_split = Subset(train_dataset, val_indices)

# set batches sizes
train_batch_size = 512
test_batch_size = 256

# Define dataloader objects that help to iterate over batches and samples for
# training, validation and testing
train_batches = DataLoader(train_split, batch_size=train_batch_size, shuffle=True)
val_batches = DataLoader(val_split, batch_size=train_batch_size, shuffle=True)
test_batches = DataLoader(test_dataset, batch_size=test_batch_size, shuffle=True)

In [3]:
# Adjustable parameters
num_train_batches = len(train_batches)
num_val_batches = len(val_batches)
num_test_batches = len(test_batches)
input_dim = 784  # Input dimension
output_dim = 10  # Output dimension (number of classes)
num_hidden_layers = 2  # Adjustable number of hidden layers
hidden_dim = [400, 400]  # Adjustable number of neurons in each hidden layer
learning_rate = 5e-2  # Adjustable learning rate
num_epochs = 15  # Adjustable number of epochs

In [ ]:
# Define your (As Cool As It Gets) Fully Connected Neural Network 
class ACAIGFCN(nn.Module):
    def __init__(self, input_dim, output_dim, num_hidden_layers, hidden_dim):
        super(ACAIGFCN, self).__init__()
        layers = []
        prev_dim = input_dim
        for dim in hidden_dim:
            layers.append(nn.Linear(prev_dim, dim))
            layers.append(nn.ReLU())
            prev_dim = dim
        layers.append(nn.Linear(prev_dim, output_dim))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [50]:
# Initialize neural network model with input, output and hidden layer dimensions
model = ACAIGFCN(input_dim, output_dim, num_hidden_layers, hidden_dim)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Iterate over epochs, batches with progress bar and train+ validate the ACAIGFCN
# Track the loss and validation accuracy
# ACAIGFCN Training
train_losses = []
val_accuracies = []
for epoch in range(num_epochs):
    # Set model into training mode
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm.tqdm(train_batches, desc=f'Epoch {epoch+1}/{num_epochs}'):
        optimizer.zero_grad()
        outputs = model(inputs.view(inputs.size(0), -1))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_split)
    train_losses.append(epoch_loss)

    # ACAIGFCN Validation
    model.eval()
    correct = 0
    total = 0
    # Telling PyTorch we aren't passing inputs to network for training purpose
    with torch.no_grad():
        for inputs, labels in val_batches:
            outputs = model(inputs.view(inputs.size(0), -1))
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_accuracy = correct / total
    val_accuracies.append(val_accuracy)
    # Record accuracy for the epoch; print training loss, validation accuracy
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss}, Validation Accuracy: {val_accuracy * 100:.2f}%')

# Testing
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for inputs, labels in test_batches:
        outputs = model(inputs.view(inputs.size(0), -1))
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
test_accuracy = test_correct / test_total
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')



Epoch 1/15:   0%|          | 0/106 [00:00<?, ?it/s]

Epoch 1/15: 100%|██████████| 106/106 [00:02<00:00, 40.66it/s]


Epoch 1/15, Training Loss: 1.0389211749500697, Validation Accuracy: 77.22%


Epoch 2/15: 100%|██████████| 106/106 [00:02<00:00, 46.56it/s]


Epoch 2/15, Training Loss: 0.6003602162643715, Validation Accuracy: 80.83%


...

Epoch 13/15, Training Loss: 0.3452149110811728, Validation Accuracy: 87.43%


Epoch 14/15: 100%|██████████| 106/106 [00:02<00:00, 47.20it/s]


Epoch 14/15, Training Loss: 0.33604657580234387, Validation Accuracy: 86.48%


Epoch 15/15: 100%|██████████| 106/106 [00:02<00:00, 46.56it/s]


Epoch 15/15, Training Loss: 0.32954068407305964, Validation Accuracy: 87.27%
Test Accuracy: 86.01%


In [ ]:
# Plotting training loss curve
plt.plot(train_losses)
plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.title('Training Loss Curve')
plt.show()

In [ ]:
# Plot training loss curve and validation accuracy curve
plt.plot(train_losses, label='Training Loss')
plt.plot(val_accuracies, label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.title('Training Loss and Validation Accuracy')
plt.legend()
plt.savefig('582hw4f1.pdf')
plt.show()

In [57]:
# Now, define a function based on the above model that allows us to quickly try out different optimizer
def train_model(optimizer, learning_rate, num_epochs=15):
    # Initialize model
    model = ACAIGFCN(input_dim, output_dim, num_hidden_layers, hidden_dim)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer(model.parameters(), lr=learning_rate)

    # Training loop
    train_losses = []
    val_accuracies = []
    for epoch in range(num_epochs):
        # Training
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm.tqdm(train_batches, desc=f'Epoch {epoch+1}/{num_epochs}'):
            optimizer.zero_grad()
            outputs = model(inputs.view(inputs.size(0), -1))
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_split)
        train_losses.append(epoch_loss)

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_batches:
                outputs = model(inputs.view(inputs.size(0), -1))
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_accuracy = correct / total
        val_accuracies.append(val_accuracy)

    # Testing
    model.eval()
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for inputs, labels in test_batches:
            outputs = model(inputs.view(inputs.size(0), -1))
            _, predicted = torch.max(outputs, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    test_accuracy = test_correct / test_total

    return train_losses, val_accuracies, test_accuracy

In [6]:
optimizers = {
    'SGD': optim.SGD,
    'RMSProp': optim.RMSprop,
    'Adam': optim.Adam
}

learning_rates = [0.001, 0.01, 0.1]

In [59]:
results = {}

for optimizer_name, optimizer_func in optimizers.items():
    for lr in learning_rates:
        print(f'Training with {optimizer_name} optimizer and learning rate {lr}...')
        train_losses, val_accuracies, test_accuracy = train_model(optimizer_func, lr)
        results[(optimizer_name, lr)] = {
            'train_losses': train_losses,
            'val_accuracies': val_accuracies,
            'test_accuracy': test_accuracy
        }

Training with SGD optimizer and learning rate 0.001...


Epoch 15/15: 100%|██████████| 106/106 [00:02<00:00, 47.10it/s]


Training with SGD optimizer and learning rate 0.01...


Epoch 15/15: 100%|██████████| 106/106 [00:02<00:00, 47.16it/s]


Training with SGD optimizer and learning rate 0.1...


...

Epoch 15/15: 100%|██████████| 106/106 [00:02<00:00, 41.14it/s]


Training with Adam optimizer and learning rate 0.01...


Epoch 15/15: 100%|██████████| 106/106 [00:02<00:00, 43.76it/s]


Training with Adam optimizer and learning rate 0.1...


Epoch 15/15: 100%|██████████| 106/106 [00:02<00:00, 39.85it/s]


In [ ]:
# Plot and compare the performance of different optimizers
plt.figure(figsize=(12, 8))
for (optimizer_name, lr), result in results.items():
    plt.plot(result['train_losses'], label=f'{optimizer_name} (lr={lr:.3f})')

plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.title('Training Loss Curves for Different Optimizers and Learning Rates')
plt.legend()
plt.show()

plt.figure(figsize=(12, 8))
for (optimizer_name, lr), result in results.items():
    plt.plot(result['val_accuracies'], label=f'{optimizer_name} (lr={lr:.3f})')

plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy Curves for Different Optimizers and Learning Rates')
plt.legend()
plt.savefig('582hw4f2.pdf')
plt.show()

# Print test accuracies
print('\nTest Accuracies:')
for (optimizer_name, lr), result in results.items():
    print(f'{optimizer_name} (lr={lr:.3f}): {result["test_accuracy"] * 100:.2f}%')


In [3]:
# Define a new FCN that takes dropout probability as one of the parameters
class ACAIGFCNWithDropout(nn.Module):
    def __init__(self, input_dim, output_dim, num_hidden_layers, hidden_dim, dropout_prob=0.5):
        super(ACAIGFCNWithDropout, self).__init__()
        layers = []
        prev_dim = input_dim
        for dim in hidden_dim:
            layers.append(nn.Linear(prev_dim, dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_prob))
            prev_dim = dim
        layers.append(nn.Linear(prev_dim, output_dim))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


In [4]:
def train_model_with_dropout(optimizer, learning_rate, num_epochs=15, dropout_prob=0.5):
    # Initialize model with dropout regularization
    model = ACAIGFCNWithDropout(input_dim, output_dim, num_hidden_layers, hidden_dim, dropout_prob)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer(model.parameters(), lr=learning_rate)

    # Training loop
    train_losses = []
    val_accuracies = []
    for epoch in range(num_epochs):
        # Training
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm.tqdm(train_batches, desc=f'Epoch {epoch+1}/{num_epochs}'):
            optimizer.zero_grad()
            outputs = model(inputs.view(inputs.size(0), -1))
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_split)
        train_losses.append(epoch_loss)

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_batches:
                outputs = model(inputs.view(inputs.size(0), -1))
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_accuracy = correct / total
        val_accuracies.append(val_accuracy)

    # Testing
    model.eval()
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for inputs, labels in test_batches:
            outputs = model(inputs.view(inputs.size(0), -1))
            _, predicted = torch.max(outputs, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    test_accuracy = test_correct / test_total

    return train_losses, val_accuracies, test_accuracy

In [7]:
dropout_results = {}

for optimizer_name, optimizer_func in optimizers.items():
    for lr in learning_rates:
        print(f'Training with {optimizer_name} optimizer and learning rate {lr}...')
        dropout_train_losses, dropout_val_accuracies, dropout_test_accuracy = train_model_with_dropout(optimizer_func, lr)
        dropout_results[(optimizer_name, lr)] = {
            'train_losses': dropout_train_losses,
            'val_accuracies': dropout_val_accuracies,
            'test_accuracy': dropout_test_accuracy
        }

Training with SGD optimizer and learning rate 0.001...


Epoch 15/15: 100%|██████████| 106/106 [00:02<00:00, 42.90it/s]


Training with SGD optimizer and learning rate 0.01...


Epoch 15/15: 100%|██████████| 106/106 [00:02<00:00, 41.35it/s]


Training with SGD optimizer and learning rate 0.1...


...

Epoch 15/15: 100%|██████████| 106/106 [00:02<00:00, 40.34it/s]


Training with Adam optimizer and learning rate 0.01...


Epoch 15/15: 100%|██████████| 106/106 [00:02<00:00, 40.70it/s]


Training with Adam optimizer and learning rate 0.1...


Epoch 15/15: 100%|██████████| 106/106 [00:02<00:00, 39.66it/s]


In [ ]:
# Plotting
plt.figure(figsize=(12, 8))
for (optimizer_name, lr), result in dropout_results.items():
    plt.plot(result['train_losses'], label=f'{optimizer_name} (lr={lr:.3f})')

plt.xlabel('Epoch')
plt.ylabel('Training Loss')
plt.title('Training Loss Curves with Dropout Regularization')
plt.legend()
plt.savefig('582hw4f4.pdf')
plt.show()

plt.figure(figsize=(12, 8))
for (optimizer_name, lr), result in dropout_results.items():
    plt.plot(result['val_accuracies'], label=f'{optimizer_name} (lr={lr:.3f})')

plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy Curves with Dropout Regularization')
plt.legend()
plt.savefig('582hw4f3.pdf')
plt.show()

# Print test accuracies
print('\nTest Accuracies with Dropout Regularization:')
for (optimizer_name, lr), result in dropout_results.items():
    print(f'{optimizer_name} (lr={lr:.3f}): {result["test_accuracy"] * 100:.2f}%')

In [9]:
# Define a new FCN to test different initializations
class ACAIGFCNWithInitialization(nn.Module):
    def __init__(self, input_dim, output_dim, num_hidden_layers, hidden_dim, initialization):
        super(ACAIGFCNWithInitialization, self).__init__()
        layers = []
        prev_dim = input_dim
        for dim in hidden_dim:
            layer = nn.Linear(prev_dim, dim)
            initialization(layer.weight.data)
            layers.append(layer)
            layers.append(nn.ReLU())
            prev_dim = dim
        layers.append(nn.Linear(prev_dim, output_dim))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)


In [10]:
def train_model_with_initialization(optimizer, learning_rate, initialization, num_epochs=15):
    # Initialize model with specified initialization
    model = ACAIGFCNWithInitialization(input_dim, output_dim, num_hidden_layers, hidden_dim, initialization)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer(model.parameters(), lr=learning_rate)

    # Training loop
    train_losses = []
    val_accuracies = []
    for epoch in range(num_epochs):
        # Training
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm.tqdm(train_batches, desc=f'Epoch {epoch+1}/{num_epochs}'):
            optimizer.zero_grad()
            outputs = model(inputs.view(inputs.size(0), -1))
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_split)
        train_losses.append(epoch_loss)

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_batches:
                outputs = model(inputs.view(inputs.size(0), -1))
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_accuracy = correct / total
        val_accuracies.append(val_accuracy)

    # Testing
    model.eval()
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for inputs, labels in test_batches:
            outputs = model(inputs.view(inputs.size(0), -1))
            _, predicted = torch.max(outputs, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    test_accuracy = test_correct / test_total

    return train_losses, val_accuracies, test_accuracy

In [13]:
initializations = {
    'Random Normal': nn.init.normal_,
    'Xavier Normal': nn.init.xavier_normal_,
    'Kaiming Uniform': nn.init.kaiming_uniform_
}

results_initialization = {}
# Given my laptop capacity, I modified the baseline configuration and picked Adam optimizer with learning rate 0.001 to test the initializations
# as they are the best performer
for initialization_name, initialization_func in initializations.items():
    print(f'Training with Adam optimizer, learning rate 0.001, and initialization {initialization_name}...')
    train_losses, val_accuracies, test_accuracy = train_model_with_initialization(optim.Adam, 0.001, initialization_func)
    results_initialization[(optim.Adam, 0.001, initialization_name)] = {
        'train_losses': train_losses,
        'val_accuracies': val_accuracies,
        'test_accuracy': test_accuracy
    }


Training with Adam optimizer, learning rate 0.001, and initialization Random Normal...


Epoch 15/15: 100%|██████████| 106/106 [00:02<00:00, 45.94it/s]


Training with Adam optimizer, learning rate 0.001, and initialization Xavier Normal...


Epoch 15/15: 100%|██████████| 106/106 [00:02<00:00, 45.69it/s]


Training with Adam optimizer, learning rate 0.001, and initialization Kaiming Uniform...


Epoch 15/15: 100%|██████████| 106/106 [00:02<00:00, 44.74it/s]


In [18]:
for initialization_name, initialization_func in initializations.items():
    print(f'Test Accuracy with {initialization_name} Initialization: {test_accuracy * 100:.2f}%')

Test Accuracy with Random Normal Initialization: 88.96%
Test Accuracy with Xavier Normal Initialization: 88.96%
Test Accuracy with Kaiming Uniform Initialization: 88.96%


In [4]:
# # Define a new FCN to test batch normalization, not considering initializations
# class ACAIGFCNWithNormalization(nn.Module):
#     def __init__(self, input_dim, output_dim, num_hidden_layers, hidden_dim):
#         super(ACAIGFCNWithNormalization, self).__init__()
#         layers = []
#         prev_dim = input_dim
#         for dim in hidden_dim:
#             layers.append(nn.Linear(prev_dim, dim))
#             layers.append(nn.BatchNorm1d(dim))  # Batch Normalization
#             layers.append(nn.ReLU())
#             prev_dim = dim
#         layers.append(nn.Linear(prev_dim, output_dim))
#         self.model = nn.Sequential(*layers)

#     def forward(self, x):
#         return self.model(x)

In [20]:
# # Given my laptop capacity, I modified the baseline configuration and picked Adam optimizer with learning rate 0.001 as they are the best performer
# def train_model_with_normalization(num_epochs=15):
#     # Initialize model with Batch Normalization
#     model = ACAIGFCNWithNormalization(input_dim, output_dim, num_hidden_layers, hidden_dim)

#     # Loss and optimizer
#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(model.parameters(), lr=0.001)

#     # Training loop
#     train_losses = []
#     val_accuracies = []
#     for epoch in range(num_epochs):
#         # Training
#         model.train()
#         running_loss = 0.0
#         for inputs, labels in train_batches:
#             optimizer.zero_grad()
#             outputs = model(inputs.view(inputs.size(0), -1))
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#             running_loss += loss.item() * inputs.size(0)
#         epoch_loss = running_loss / len(train_split)
#         train_losses.append(epoch_loss)

#         # Validation
#         model.eval()
#         correct = 0
#         total = 0
#         with torch.no_grad():
#             for inputs, labels in val_batches:
#                 outputs = model(inputs.view(inputs.size(0), -1))
#                 _, predicted = torch.max(outputs, 1)
#                 total += labels.size(0)
#                 correct += (predicted == labels).sum().item()
#         val_accuracy = correct / total
#         val_accuracies.append(val_accuracy)

#     # Testing
#     model.eval()
#     test_correct = 0
#     test_total = 0
#     with torch.no_grad():
#         for inputs, labels in test_batches:
#             outputs = model(inputs.view(inputs.size(0), -1))
#             _, predicted = torch.max(outputs, 1)
#             test_total += labels.size(0)
#             test_correct += (predicted == labels).sum().item()
#     test_accuracy = test_correct / test_total

#     return train_losses, val_accuracies, test_accuracy

In [21]:
# # Train model with Batch Normalization
# train_losses_with_normalization, val_accuracies_with_normalization, test_accuracy_with_normalization = train_model_with_normalization()

# print(f'Test Accuracy with Batch Normalization: {test_accuracy_with_normalization * 100:.2f}%')


Test Accuracy with Batch Normalization: 88.84%


In [7]:
class ACAIGFCNWithNormalization(nn.Module):
    def __init__(self, input_dim, output_dim, num_hidden_layers, hidden_dim, initialization):
        super(ACAIGFCNWithNormalization, self).__init__()
        layers = []
        prev_dim = input_dim
        for dim in hidden_dim:
            layers.append(nn.Linear(prev_dim, dim))
            layers.append(nn.BatchNorm1d(dim))  # Batch Normalization
            layers.append(nn.ReLU())
            prev_dim = dim
        layers.append(nn.Linear(prev_dim, output_dim))
        self.model = nn.Sequential(*layers)

        # Initialization
        for m in self.model.modules():
            if isinstance(m, nn.Linear):
                initialization(m.weight)  

    def forward(self, x):
        return self.model(x)

In [9]:
def train_model_with_normalization(optimizer, learning_rate, initialization, num_epochs=15):
    # Initialize model with specified initialization
    model = ACAIGFCNWithNormalization(input_dim, output_dim, num_hidden_layers, hidden_dim, initialization)

    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optimizer(model.parameters(), lr=learning_rate)

    # Training loop
    train_losses = []
    val_accuracies = []
    for epoch in range(num_epochs):
        # Training
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm.tqdm(train_batches, desc=f'Epoch {epoch+1}/{num_epochs}'):
            optimizer.zero_grad()
            outputs = model(inputs.view(inputs.size(0), -1))
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / len(train_split)
        train_losses.append(epoch_loss)

        # Validation
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_batches:
                outputs = model(inputs.view(inputs.size(0), -1))
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_accuracy = correct / total
        val_accuracies.append(val_accuracy)

    # Testing
    model.eval()
    test_correct = 0
    test_total = 0
    with torch.no_grad():
        for inputs, labels in test_batches:
            outputs = model(inputs.view(inputs.size(0), -1))
            _, predicted = torch.max(outputs, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    test_accuracy = test_correct / test_total

    return train_losses, val_accuracies, test_accuracy


In [10]:
initializations = {
    'Random Normal': nn.init.normal_,
    'Xavier Normal': nn.init.xavier_normal_,
    'Kaiming Uniform': nn.init.kaiming_uniform_
}
results_initialization = {}

for initialization_name, initialization_func in initializations.items():
    print(f'Training with Adam optimizer, learning rate 0.001, initialization {initialization_name}, and Batch Normalization...')
    train_losses, val_accuracies, test_accuracy = train_model_with_normalization(optim.Adam, 0.001, initialization_func)
    results_initialization[(initialization_name)] = {
        'train_losses': train_losses,
        'val_accuracies': val_accuracies,
        'test_accuracy': test_accuracy
    }
    print(f'Test Accuracy with {initialization_name} Initialization: {test_accuracy * 100:.2f}%')


Training with Adam optimizer, learning rate 0.001, initialization Random Normal, and Batch Normalization...


Epoch 15/15: 100%|██████████| 106/106 [00:02<00:00, 43.74it/s]


Test Accuracy with Random Normal Initialization: 84.78%
Training with Adam optimizer, learning rate 0.001, initialization Xavier Normal, and Batch Normalization...


Epoch 15/15: 100%|██████████| 106/106 [00:02<00:00, 45.24it/s]


Test Accuracy with Xavier Normal Initialization: 88.75%
Training with Adam optimizer, learning rate 0.001, initialization Kaiming Uniform, and Batch Normalization...


Epoch 15/15: 100%|██████████| 106/106 [00:02<00:00, 43.02it/s]


Test Accuracy with Kaiming Uniform Initialization: 88.50%
